In [ ]:
# import YOLO and create model
from ultralytics import YOLO
import supervision as sv
import numpy as np
from sklearn.cluster import KMeans

MODEL = 'yolov8x.pt'

model = YOLO(MODEL)
model.fuse()

 82%|████████▏ | 107M/131M [00:09<00:02, 11.4MB/s] 

In [1]:
# Configuration
CAMERA_ID = "I94-US20-35"
#CAMERA_ID =  I65-53-1
#CAMERA_ID =  I65-53-2
VIDEO_ID = 4 # 1 - 4

# settings for counting line
# I94-US20-35
LINE_START = sv.Point(350, 1000)
LINE_END = sv.Point(1400, 600)
# I65-53-1
#LINE_START = sv.Point(350, 1000)
#LINE_END = sv.Point(1400, 600)
# I65-53-1
#LINE_START = sv.Point(350, 1000)
#LINE_END = sv.Point(1400, 600)

SOURCE_VIDEO_PATH = f"../data/{CAMERA_ID}.{VIDEO_ID}.mp4"
TARGET_VIDEO_PATH = f'../demo/results.{CAMERA_ID}.{VIDEO_ID}.mp4'

# dict mapping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest : car, motorcycle, and truck
selected_classes = [2, 3, 7]

# get video info
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# MPH_CONVERSION_FACTOR = (feet/frame) * (frames/second) * (seconds/minute) * (minutes/hour) * (mile/feet)
MPH_CONVERSION_FACTOR = 30 * 60 * 60 / 5280

NameError: name 'sv' is not defined

In [3]:
# Matt

In [4]:
# Travis
class Coordinate:
    def __init__(self, x, y):
        self.x = int(x)
        self.y = int(y)

    def origin() -> Coordinates:
        return new Coordinate(0, 0)

class Automobile:
    def __init__(self, class_id, tracker_id):
        self.class = class_id
        self.tracker = tracker_id
        self.coordinates = None


    def update_location(coord):
        if self.coordinates == None:
            self.coordinates = np.ndarray(coord)
        elif self.coordinates.shape[0] < 4:
            self.coordinates = np.concatinate((self.coordinates, coord))
        else:
            self.coordinates = np.concatinate((self.coordinates[1:3], coord))

    def get_speed(pixel_scale: float) -> float:
        if self.coordinates == None:
            return 0.0
        if self.coordinates.shape[0] == 1:
            return 0.0

        size = self.coordinates.shape[0]
        startpoint = self.coordinates[0]
        endpoint = self.coordinates[size - 1]

        # Taking the average distance of size frames
        distance = np.linalg.norm(startpoint-endpoint) / (size - 1)

        speed = distance * pixel_scale * MPH_CONVERSION_FACTOR
        return speed




In [5]:
# Yash

In [6]:
# Rui

In [7]:
# Tao

# Calculate the average color of an image region
def get_average_color(image):
    return np.mean(image, axis=(0, 1)).astype(int)

def get_color_name(rgb_color):
    color_ranges = {
        'Red': ([100, 0, 0], [255, 150, 100]),
        'Green': ([0, 150, 0], [150, 255, 255]),
        'Black': ([0, 0, 0], [150, 150, 150]),
        'Blue': ([0, 0, 200], [50, 50, 255]),
        'Yellow': ([150, 50, 0], [255, 255, 100]),
        'White': ([150, 150, 200], [255, 255, 255]),
        'Gray': ([100, 100, 100], [200, 200, 200]),
    }

    for color_name, (lower, upper) in color_ranges.items():
        if all(lower[i] <= rgb_color[i] <= upper[i] for i in range(3)):
            return color_name
    return "Unknown"

In [8]:
# This is where all the frame processing happens and is looped through

# for general tracking
# ie update location
def update_automobiles(detections):
    for detection in detections:
        # find object with that id, if not exists create one
        # update with latest coordinates


# for when they cross the line
# ie calculate speed and color
def update_lz_automobiles(lz_detections):
    for detection in lz_detections:
        # calculate color
        # calculate speed


# Call back to process each frame
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # Wow, not much to do here really except add on what we need and save a file that lists the results, ie, #5 red truck in or something
    # model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # only consider class id from selected_classes define above
    detections = detections[np.isin(detections.class_id, selected_classes)]
    # tracking detections
    detections = byte_tracker.update_with_detections(detections)
    # update line counter
    lz_results = lz_detections = line_zone.trigger(detections)

    update_automobiles(detections)
    update_lz_automobiles(lz_detections[0])

    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]
    annotated_frame = trace_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )
    annotated_frame=label_annotator.annotate(
        scene=annotated_frame,
        detections=detections,
        labels=labels)


    # return frame with box and line annotated result
    return  line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

In [9]:
# Initialization of objects used


# create bytetracker instance
# defaults track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)
byte_tracker = sv.ByteTrack()

# create videoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# create LineZone instance
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# create instance of BoxAnnotator
label_annotator = sv.LabelAnnotator(text_position=sv.Position.TOP_CENTER)

# create instance of TraceAnnotator
trace_annotator = sv.TraceAnnotator(trace_length=50)

# create LineZoneAnnotator instance
line_zone_annotator = sv.LineZoneAnnotator()

In [ ]:
# Run Me Traffic


sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)

# itterate through all the automobiles and write output to file
